In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('data/ckd_clean.csv')
df

,age,blood_pressure,specific_gravity,albumin,sugar,abnormal_red_blood_cells,abnormal_pus_cell,pus_cell_clumps,bacteria,blood_glucose_random,...,haemoglobin,packed_cell_volume,white_blood_cell_count,red_blood_cell_count,hypertension,diabetes_mellitus,coronary_artery_disease,poor_appetite,peda_edema,aanemia
0,48.0,80.0,1.020,1.0,0.0,0.166667,0.0,0.0,0.0,121.000000,...,15.4,44.0,7800.0,5.200000,1.0,1.0,0.0,0.0,0.0,0.0
1,7.0,50.0,1.020,4.0,0.0,0.083333,0.0,0.0,0.0,110.958333,...,11.3,38.0,6000.0,5.329167,0.0,0.0,0.0,0.0,0.0,0.0
2,62.0,80.0,1.010,2.0,3.0,0.000000,0.0,0.0,0.0,423.000000,...,9.6,31.0,7500.0,3.987500,0.0,1.0,0.0,1.0,0.0,1.0
3,48.0,70.0,1.005,4.0,0.0,0.000000,1.0,1.0,0.0,117.000000,...,11.2,32.0,6700.0,3.900000,1.0,0.0,0.0,1.0,1.0,1.0
4,51.0,80.0,1.010,2.0,0.0,0.000000,0.0,0.0,0.0,106.000000,...,11.6,35.0,7300.0,4.600000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,0.000000,0.0,0.0,0.0,140.000000,...,15.7,47.0,6700.0,4.900000,0.0,0.0,0.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,0.000000,0.0,0.0,0.0,75.000000,...,16.5,54.0,7800.0,6.200000,0.0,0.0,0.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,0.000000,0.0,0.0,0.0,100.000000,...,15.8,49.0,6600.0,5.400000,0.0,0.0,0.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,0.000000,0.0,0.0,0.0,114.000000,...,14.2,51.0,7200.0,5.900000,0.0,0.0,0.0,0.0,0.0,0.0
